# HW 5 Group 7 

- First download the dataset ("holy_grail.txt") from Stud.IP and put it on the same level as this notebook

## General Setup

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
# General tensorflow settings
config = tf.ConfigProto()
# Use GPU in incremental mode (is ignored on CPU version)
config.gpu_options.allow_growth=True
# Add config=config in every tf.Session() -> tf.Session(config=config)

# We have to enable eager execution to loop over the input
#tf.enable_eager_execution() # Not working with placeholders, alternative: tf.map_fn or tf.unstack

## Data preparation and visualization

In [3]:
# First read in the data
text = open("holy_grail.txt", 'r').read()
#text = text[:1000]
text_length = len(text)

In [4]:
# We create our vocabulary
vocab = set(text)
vocab_len = len(vocab)
# Create the index for each character and vice versa
char_to_pos = {char:pos for pos, char in enumerate(vocab)}
pos_to_char = {pos:char for pos, char in enumerate(vocab)}

# Print some properties of our data
print("The text contains {} letters and {} unique characters".format(text_length, vocab_len))

# Transform each character to the position of that character in the vocabulary
text_number = [char_to_pos[char] for char in text]

The text contains 60061 letters and 78 unique characters


In [5]:
# Create the chunk with a sub sequence length of 25
chunks = [text_number[i:i+25] for i in range(text_length-25)]
# Construct the input sequences and the target sequences
input_sequences = chunks[:-1]
target_sequences = chunks[1:]

In [6]:
# Create the dataset for tensorflow

# First we reset the graph
tf.reset_default_graph()

# Now we use the tf.data library to create a tensorflow dataset
training_dataset = tf.data.Dataset.from_tensor_slices((input_sequences, target_sequences))

In [7]:
# Create the iterator 

# First we create the iterator
iterator = tf.data.Iterator.from_structure(training_dataset.output_types,
                                           training_dataset.output_shapes)

# We prepare the initializer operations for the training
training_init_op = iterator.make_initializer(training_dataset)

# We name the get_next method of the iterator to use it as a shortcut
next_batch = iterator.get_next()

## The model

In [8]:
# First we will format the data in the correct way

# We define our input data and the corresponding target data and transform it to one hot encoding
input_data = tf.one_hot(next_batch[0], vocab_len)
target_data = tf.one_hot(next_batch[1], vocab_len)

print(input_data, target_data)

hidden_len = 100
# Create the placeholder for the hidden state
rem_hidden_state = tf.placeholder(tf.float32, shape=(1,hidden_len))

print(rem_hidden_state)

Tensor("one_hot:0", shape=(25, 78), dtype=float32) Tensor("one_hot_1:0", shape=(25, 78), dtype=float32)
Tensor("Placeholder:0", shape=(1, 100), dtype=float32)


In [9]:
# Now we build our RNN

# We have to define our lists
hidden_states = []
logits = []

# We define all our weights and biases
w_xh = tf.Variable(tf.random_normal([vocab_len, hidden_len]))
w_hh = tf.Variable(tf.random_normal([hidden_len, hidden_len]))
b_h = tf.Variable(tf.random_normal([1, hidden_len]))
w_hy = tf.Variable(tf.random_normal([hidden_len, vocab_len]))
b_y = tf.Variable(tf.random_normal([1, vocab_len]))

# We define the for_loop as a function 
#def forward_steps(char):
for char in tf.unstack(input_data):
    char = tf.reshape(char, [1, vocab_len])
    new_hidden_state = tf.tanh(char @ w_xh + rem_hidden_state @ w_hh + b_h)
    logit = new_hidden_state @ w_hy + b_y
    hidden_states.append(new_hidden_state)
    logits.append(logit)

# Fancy for loop
# tf.map_fn(forward_steps, input_data)

In [10]:
# Read out hidden states?
rem_hidden = hidden_states[-1]

# Read out logits of last iteration
final_logits = logits[-1]
output_softmax = tf.nn.softmax(final_logits)

# Loss is mean of the cross entropies in each time step
outputs = tf.concat(logits, axis=0)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=target_data, logits=outputs)
loss = tf.reduce_mean(cross_entropy)

# Optimizer
# We create an optimizer
optimizer = tf.train.AdamOptimizer()
# And tell the optimizer that it should minimize the loss
training_step = optimizer.minimize(loss)

In [11]:
# Finally we can train our model

# First we specify the number of epochs
epochs = 10

# We safely create our tensorflow session and pass our config parameters (for correct GPU usage, if GPU available)
with tf.Session(config=config) as sess:
    
    # We initialize our variables
    sess.run(tf.global_variables_initializer())
    
    # We define a step counter (for the summaries)
    global_step = 0
    
    # We will run our training as often as specified in epochs
    for ep in range(epochs):
        
        # Training Part
        
        # We initialize the hidden state with zeros in the beginning of each new epoch
        hidden_state = np.zeros((1, hidden_len))
        
        # We have to load the training data into the iterator
        sess.run(training_init_op)
        
        # We have to loop over all our batches in every epoch
        while True:
            try:
                # print(hidden_state.shape, hidden_state.dtype)
                # We train with one batch and read the summary and save it in the variable summary
                _, hidden_state, loss_val = sess.run([training_step, rem_hidden, loss], feed_dict={rem_hidden_state: hidden_state})
                # print(hidden_state)       
                # We update our step counter
                global_step += 1
                
            # After we finished all batches, we catch the OutOfRangeErrpr and break
            except tf.errors.OutOfRangeError:
                break
                
        # Print loss after each epoch
        print("\n----\nEp: {}, Step: {}, Loss: {}\n----\n".format(ep, global_step, loss_val))
        
        # Sampling
        sample_length = 200
        random_sub_sequences = [input_sequences[np.random.choice(len(input_sequences))]]
        random_fake_targets = [target_sequences[np.random.choice(len(target_sequences))]]
        #input_data = tf.one_hot(next_batch[0], vocab_len)
        #target_data = tf.one_hot(next_batch[1], vocab_len)
        sample_outputs = []
        
        for t in range(sample_length):
            sample_dataset = tf.data.Dataset.from_tensor_slices((random_sub_sequences, random_fake_targets))
            sess.run(iterator.make_initializer(sample_dataset))
            sample_output_softmax, hidden_state = sess.run([output_softmax, rem_hidden], feed_dict={rem_hidden_state: hidden_state})
            sample_output = np.random.choice(vocab_len, p=sample_output_softmax.ravel())
            sample_outputs.append(sample_output)
            # Delete the first character and append the newly sampled one
            random_sub_sequences[0] = random_sub_sequences[0][1:] + [sample_output]
        
        # Print the sample
        print(''.join(pos_to_char[pos] for pos in sample_outputs))           

----
Ep: 0, Step: 60035, Loss: 2.577828884124756
----

  alannlcartlapal
Ssk, akdktg
 t aetle t!
 atond
Sg
 RTHARDIbrtyS atamst
 at!  afs
 amk,
 Skl, yomy!
 An
 bbae alaaboot I!
 Se!
Stat!
Staat!
 SttattstAy, t
 ld!
St ted
Sy AR: y
 totarsthabfyofot taot!
----
Ep: 1, Step: 120070, Loss: 2.284122943878174
----

!
 t ARDIy!
 billdt!




   p!
 yold!
  y MI  Sntgang g St St AHUSnd rttanat Imlt    lory!
 tklk!
 R: ft Swe RDIyoandyt,  mat byoflstat tan RTHUR: tttat SlySmt  ofaftd!
 R: t
  R: ndt
 my Att amatdt  
----
Ep: 2, Step: 180105, Loss: 2.4048447608947754
----

e y!
 abaondtot Sbrininanird  ARSbuld g  aER: Standtad  ARTHURTHURTHUAR:  AR: AR: Yof RDyys
  R: Sk Snaby R: y
 ARTHUR: Smang e
 St St!
 :  tanat,  yly
 y, Stet
 Spanane t aaticet

 St
 RTHUR: f
 HURT
----
Ep: 3, Step: 240140, Loss: 2.527308702468872
----

at  AERTHUR: S: bk S:  S: at,  S: ARy otyS: att!
ScyAR: AR: ARy!
 yotamyotet!
 HUR: ARTHURTHURTHURouraUR: AR: y
 ARTHURTbtfl, AR:  S: Otatth and

 S: TSt!
 I IR: S: arrat
 S